In [ ]:
# STEP 1: Install Required Libraries
!pip install kagglehub

# STEP 2: Import Libraries
import kagglehub
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# STEP 3: Download Dataset from KaggleHub
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
print("✅ Dataset Path:", path)

# STEP 4: Define Directory Paths
train_dir = os.path.join(path, "chest_xray/train")
val_dir = os.path.join(path, "chest_xray/val")
test_dir = os.path.join(path, "chest_xray/test")

# STEP 5: Image Preprocessing (Rescale + Resize)
IMG_SIZE = (150, 150)  # You can change to (224, 224) for higher quality
BATCH_SIZE = 32

datagen = ImageDataGenerator(rescale=1. / 255)

train_gen = datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

val_gen = datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

test_gen = datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

print("✅ Image preprocessing completed and generators are ready!")


In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Build Xception model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze pretrained layers

model_xception = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model_xception.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history_xception = model_xception.fit(train_gen, validation_data=val_gen, epochs=10)

# Evaluate on test data
loss, acc = model_xception.evaluate(test_gen)
print(f"✅ Xception Test Accuracy: {acc:.4f}")

# Accuracy graph
plt.figure(figsize=(8, 5))
plt.plot(history_xception.history['accuracy'], label='Train Accuracy')
plt.plot(history_xception.history['val_accuracy'], label='Validation Accuracy')
plt.title('Xception Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Save model
model_xception.save("/content/pneumonia_cnn.h5")
